In [3]:
import torch

In [4]:
torch.__version__

'2.5.1+cu124'

In [5]:
torch.cuda.is_available()

True

In [13]:
!nvidia-smi

Mon Jul 14 05:57:08 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:09.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      5MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
from torch.backends import cudnn

x = torch.normal(0, 1, (1, 2))
x.requires_grad = True
print(cudnn.is_acceptable(x))

x = x.cuda()
print(cudnn.is_acceptable(x))

False
True


In [35]:
import time

torch.manual_seed(0)
x_cpu = torch.randn(64, 3, 224, 224)
conv_cpu = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)

# CPU 测试
start = time.time()
for _ in range(100):
    y = conv_cpu(x_cpu)
print("CPU:", time.time() - start)

# 准备 GPU 输入
x_gpu = x_cpu.cuda()

# GPU + cuDNN 关闭
torch.backends.cudnn.enabled = False
conv_gpu_nocudnn = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1).cuda()
torch.cuda.synchronize()
start = time.time()
for _ in range(100):
    y = conv_gpu_nocudnn(x_gpu)
torch.cuda.synchronize()
print("GPU without cuDNN:", time.time() - start)

# GPU + cuDNN 开启
torch.backends.cudnn.enabled = True
conv_gpu_cudnn = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1).cuda()
torch.cuda.synchronize()
start = time.time()
for _ in range(100):
    y = conv_gpu_cudnn(x_gpu)
torch.cuda.synchronize()
print("GPU with cuDNN:", time.time() - start)

CPU: 27.93531894683838
GPU without cuDNN: 1.4835753440856934
GPU with cuDNN: 1.196150302886963


In [38]:
# 原始张量：shape = (2, 3)
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("原始张量:\n", x)


原始张量:
 tensor([[1, 2, 3],
        [4, 5, 6]])


In [39]:
# ✅ 1. reshape / view：改变形状，不改变数据内容
x_reshaped = x.reshape(3, 2)  # 推荐写法
print("\nreshape 到 (3, 2):\n", x_reshaped)

x_view = x.view(-1)  # 展平成一维
print("\nview 展平为一维:\n", x_view)


reshape 到 (3, 2):
 tensor([[1, 2],
        [3, 4],
        [5, 6]])

view 展平为一维:
 tensor([1, 2, 3, 4, 5, 6])


In [40]:
# ✅ 2. unsqueeze：增加维度
x1d = torch.tensor([1, 2, 3])  # shape=(3,)

x_unsq0 = x1d.unsqueeze(0)  # shape=(1, 3)
x_unsq1 = x1d.unsqueeze(1)  # shape=(3, 1)

print("\nunsqueeze 添加第0维 (1,3):\n", x_unsq0)
print("unsqueeze 添加第1维 (3,1):\n", x_unsq1)


unsqueeze 添加第0维 (1,3):
 tensor([[1, 2, 3]])
unsqueeze 添加第1维 (3,1):
 tensor([[1],
        [2],
        [3]])


In [41]:
# ✅ 3. squeeze：去除尺寸为1的维度
x_squeeze_input = torch.tensor([[[1], [2], [3]]])  # shape=(1,3,1)
x_squeezed = x_squeeze_input.squeeze()  # shape=(3,)
print("\nsqueeze 去除尺寸为1的维度:\n", x_squeezed)


squeeze 去除尺寸为1的维度:
 tensor([1, 2, 3])


In [42]:
# ✅ 4. transpose：交换两个维度
x_transposed = x.transpose(0, 1)  # 交换第0维和第1维：shape=(3,2)
print("\ntranspose 交换维度:\n", x_transposed)


transpose 交换维度:
 tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [43]:
# ✅ 5. permute：对所有维度重新排序
x3d = torch.randn(2, 3, 4)  # shape=(2,3,4)
x_permuted = x3d.permute(2, 0, 1)  # shape=(4,2,3)
print("\npermute 重排维度 (2,3,4) -> (4,2,3):\n", x_permuted.shape)


permute 重排维度 (2,3,4) -> (4,2,3):
 torch.Size([4, 2, 3])


In [44]:
# ✅ 6. flatten：展平部分维度
x2 = torch.tensor([[1, 2], [3, 4]])
x_flattened = torch.flatten(x2)  # shape=(4,)
print("\nflatten 展平为一维:\n", x_flattened)


flatten 展平为一维:
 tensor([1, 2, 3, 4])


In [45]:
# ✅ 7. resize_：**原地操作**，可能破坏原数据
x_resize = torch.tensor([1, 2, 3, 4])
x_resize.resize_(2, 2)
print("\nresize_ 原地改变形状 (注意不推荐):\n", x_resize)


resize_ 原地改变形状 (注意不推荐):
 tensor([[1, 2],
        [3, 4]])


In [51]:
a = torch.normal(0, 1, (2, 3))
b = torch.full_like(a, 7.)
# torch.addcdiv(input, tensor1, tensor2, *, value=1, out=None)
input = torch.tensor(3.)
torch.addcdiv(input, a, b, value=3)
# torch.addcmul

tensor([[2.6535, 3.0759, 2.6140],
        [3.4554, 1.9575, 3.6414]])

In [52]:
torch.neg(a)

tensor([[ 0.8086, -0.1770,  0.9006],
        [-1.0626,  2.4325, -1.4965]])

In [56]:
torch.dist(a, b, p=torch.inf)

tensor(9.4325)

In [57]:
torch.ge(a, b)

tensor([[False, False, False],
        [False, False, False]])

In [59]:
torch.le(a, b)

tensor([[True, True, True],
        [True, True, True]])

In [60]:
torch.equal(a, b)

False

In [61]:
torch.gt(a, b)

tensor([[False, False, False],
        [False, False, False]])

In [62]:
torch.lt(a, b)

tensor([[True, True, True],
        [True, True, True]])

In [68]:
print(a)
print(torch.topk(a, 2, dim=1)[0])
print(torch.topk(a, 2, dim=1)[1])

tensor([[-0.8086,  0.1770, -0.9006],
        [ 1.0626, -2.4325,  1.4965]])
tensor([[ 0.1770, -0.8086],
        [ 1.4965,  1.0626]])
tensor([[1, 0],
        [2, 0]])


In [69]:
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])

# 与标量相乘
print(a * 2)                  # tensor([2.0, 4.0, 6.0])
print(torch.mul(a, 2))        # 同上

# 逐元素乘法（element-wise）
print(a * b)                  # tensor([4.0, 10.0, 18.0])
print(torch.mul(a, b))        # 同上

tensor([2., 4., 6.])
tensor([2., 4., 6.])
tensor([ 4., 10., 18.])
tensor([ 4., 10., 18.])


In [70]:
A = torch.tensor([[1, 2], [3, 4]])
B = torch.tensor([[5, 6], [7, 8]])

# 推荐方式：使用 @ 运算符
print(A @ B)

# 使用 matmul（支持 1D/2D/高维）
print(torch.matmul(A, B))

# 使用 mm（只支持 2D）
print(torch.mm(A, B))

tensor([[19, 22],
        [43, 50]])
tensor([[19, 22],
        [43, 50]])
tensor([[19, 22],
        [43, 50]])


In [72]:
A = torch.randn(10, 2, 3)
B = torch.randn(10, 3, 4)

# 批量矩阵乘法：输出形状为 (10, 2, 4)
C = torch.bmm(A, B)
print(C)

tensor([[[ 1.2595, -0.0759,  2.7515,  1.0268],
         [-3.1586, -1.1257,  1.0849, -1.8221]],

        [[-0.8149, -1.1841,  8.0182,  2.7357],
         [ 0.0549, -0.0522, -1.0161,  0.1989]],

        [[ 2.0990,  0.3018, -0.4382, -0.5291],
         [-6.0966, -2.6868,  0.1488,  1.2637]],

        [[-0.2429,  0.0813, -0.1786,  1.6716],
         [-2.3099,  0.3499, -1.6873, -0.9331]],

        [[-4.1601, -1.7306, -0.1414,  3.4027],
         [ 0.0715, -0.5991,  0.0747, -0.8925]],

        [[-0.7883,  2.3012,  1.3500,  3.2958],
         [-0.3670,  1.7750,  1.6516,  0.2526]],

        [[-0.1083, -0.9768, -3.1181,  1.4984],
         [ 1.1062, -0.0102,  0.1982, -0.7728]],

        [[-0.4367,  1.6546, -2.4659, -2.2413],
         [-0.6002,  1.1254, -0.4258, -0.2553]],

        [[ 2.0478,  2.0182,  3.0777,  1.5817],
         [ 0.3768,  1.6207, -0.1454,  0.6417]],

        [[ 0.3362,  1.8548, -1.7134,  0.4524],
         [-0.5935, -0.4444,  0.6190,  0.0428]]])


In [73]:
x = torch.tensor([1.0, 2.0, 3.0])
y = torch.tensor([4.0, 5.0, 6.0])

# 1D 向量点积：结果是标量
print(torch.dot(x, y))  # 32.0

tensor(32.)


In [85]:
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0])

# outer product：3x2 矩阵
print(torch.outer(a, b))

tensor([[ 4.,  5.],
        [ 8., 10.],
        [12., 15.]])


In [86]:
a.unsqueeze_(1)
b.unsqueeze_(0)
print(torch.mm(a, b))

tensor([[ 4.,  5.],
        [ 8., 10.],
        [12., 15.]])


## PyTorch 与 NumPy 函数对照表

| 操作类别       | NumPy                                      | PyTorch                                       |
|----------------|---------------------------------------------|-----------------------------------------------|
| 数据类型       | `np.ndarray`                                | `torch.Tensor`                                |
|                | `np.float32`                                | `torch.float32`, `torch.float`                |
|                | `np.float64`                                | `torch.float64`, `torch.double`               |
|                | `np.int64`                                  | `torch.int64`, `torch.long`                   |
| 构建张量       | `np.array([3.2, 4.3], dtype=np.float16)`     | `torch.tensor([3.2, 4.3], dtype=torch.float16)` |
|                | `x.copy()`                                  | `x.clone()`                                   |
| 拼接           | `np.concatenate`                            | `torch.cat`                                   |
| 线性代数       | `np.dot`                                    | `torch.mm`                                    |
| 属性           | `x.ndim`                                    | `x.dim()`                                     |
|                | `x.size`                                    | `x.nelement()`                                |
| 形状操作       | `x.reshape`                                 | `x.reshape()`, `x.view()`                     |
|                | `x.flatten()`                               | `x.view(-1)`                                  |
| 类型转换       | `np.floor(x)`                               | `torch.floor(x)`, `x.floor()`                 |
| 比较运算       | `np.less(x, y)`                             | `x.lt(y)`                                     |
|                | `np.less_equal(x, y)` / `np.greater(x, y)` | `x.le(y)` / `x.gt(y)`                         |
|                | `np.equal(x, y)` / `np.not_equal(x, y)`    | `x.eq(y)` / `x.ne(y)`                         |
| 随机种子       | `np.random.seed(0)`                         | `torch.manual_seed(0)`                        |